# Human-in-the-Loop Middleware with Gradio UI - LangChain 1.0

**Module:** Interactive Approval Workflows for HR Operations

**What you'll learn:**
- 👤 Using built-in `HumanInTheLoopMiddleware`
- ⏸️ Pausing agent execution for approval
- ✅ Approving/rejecting operations
- 🔄 Resuming paused conversations
- 🎨 Building Gradio UI for approval workflows
- 💾 State persistence across interrupts

**HR Use Cases:**
- Salary updates requiring manager approval
- Employee terminations
- Promotions and role changes
- Sensitive data access
- Financial transactions

**Time:** 2-3 hours

---

## Setup: Install Dependencies

In [ ]:
!pip install --pre -U langchain langchain-openai langgraph
!pip install langgraph-checkpoint-sqlite gradio

## Setup: Imports and Configuration

In [ ]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Core imports
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware  # ⭐ Built-in!
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command
from typing import Annotated
from datetime import datetime
import json
import gradio as gr

print("✅ Setup complete!")
print("✅ Using LangChain's built-in HumanInTheLoopMiddleware")

## Setup: HR Database

In [ ]:
# HR Employee Database
EMPLOYEES = {
    "101": {
        "name": "Priya Sharma",
        "email": "priya.sharma@company.com",
        "department": "Engineering",
        "role": "Senior Developer",
        "salary": 120000,
        "status": "active"
    },
    "102": {
        "name": "Rahul Verma",
        "email": "rahul.verma@company.com",
        "department": "Engineering",
        "role": "Engineering Manager",
        "salary": 180000,
        "status": "active"
    },
    "103": {
        "name": "Anjali Patel",
        "email": "anjali.patel@company.com",
        "department": "HR",
        "role": "HR Director",
        "salary": 200000,
        "status": "active"
    }
}

# Global approval history
APPROVAL_HISTORY = []

print(f"✅ Loaded {len(EMPLOYEES)} employees")

## Setup: HR Tools (Safe and Sensitive)

In [ ]:
# SAFE TOOLS (auto-approve)
@tool
def get_employee_info(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Get employee information. SAFE operation."""
    if employee_id in EMPLOYEES:
        emp = EMPLOYEES[employee_id]
        return f"""Employee: {emp['name']}
Department: {emp['department']}
Role: {emp['role']}
Status: {emp['status']}"""
    return f"Employee {employee_id} not found"

@tool
def search_employees(department: Annotated[str, "Department name"]) -> str:
    """Search employees by department. SAFE operation."""
    results = []
    for emp_id, emp in EMPLOYEES.items():
        if emp['department'].lower() == department.lower():
            results.append(f"{emp['name']} ({emp['role']})")
    
    if results:
        return f"Employees in {department}: " + ", ".join(results)
    return f"No employees found in {department}"

# SENSITIVE TOOLS (require approval)
@tool
def update_salary(
    employee_id: Annotated[str, "Employee ID"],
    new_salary: Annotated[int, "New salary amount"]
) -> str:
    """Update employee salary. REQUIRES APPROVAL."""
    if employee_id in EMPLOYEES:
        old_salary = EMPLOYEES[employee_id]['salary']
        EMPLOYEES[employee_id]['salary'] = new_salary
        
        # Log to approval history
        APPROVAL_HISTORY.append({
            "operation": "update_salary",
            "employee": EMPLOYEES[employee_id]['name'],
            "old_value": old_salary,
            "new_value": new_salary,
            "timestamp": datetime.now().isoformat()
        })
        
        return f"✅ Salary updated for {EMPLOYEES[employee_id]['name']}: ₹{old_salary:,} → ₹{new_salary:,}"
    return f"Employee {employee_id} not found"

@tool
def promote_employee(
    employee_id: Annotated[str, "Employee ID"],
    new_role: Annotated[str, "New role title"]
) -> str:
    """Promote employee to new role. REQUIRES APPROVAL."""
    if employee_id in EMPLOYEES:
        old_role = EMPLOYEES[employee_id]['role']
        EMPLOYEES[employee_id]['role'] = new_role
        
        APPROVAL_HISTORY.append({
            "operation": "promote_employee",
            "employee": EMPLOYEES[employee_id]['name'],
            "old_value": old_role,
            "new_value": new_role,
            "timestamp": datetime.now().isoformat()
        })
        
        return f"✅ {EMPLOYEES[employee_id]['name']} promoted: {old_role} → {new_role}"
    return f"Employee {employee_id} not found"

@tool
def terminate_employee(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Terminate employee. CRITICAL - REQUIRES APPROVAL."""
    if employee_id in EMPLOYEES:
        EMPLOYEES[employee_id]['status'] = 'terminated'
        
        APPROVAL_HISTORY.append({
            "operation": "terminate_employee",
            "employee": EMPLOYEES[employee_id]['name'],
            "old_value": "active",
            "new_value": "terminated",
            "timestamp": datetime.now().isoformat()
        })
        
        return f"✅ Employee {EMPLOYEES[employee_id]['name']} has been terminated."
    return f"Employee {employee_id} not found"

print("✅ HR tools configured!")
print("   Safe: get_employee_info, search_employees")
print("   Sensitive: update_salary, promote_employee, terminate_employee")

---
# Part 1: Understanding Human-in-the-Loop

## How It Works

```
User Request
    ↓
Agent Processes
    ↓
Needs to call tool?
    ↓
Is tool sensitive? ──→ NO ──→ Execute immediately
    ↓ YES
⏸️  PAUSE & Wait for approval
    ↓
Human reviews
    ↓
✅ Approve OR ❌ Reject
    ↓
Resume execution
```

## Key Concepts

1. **Interrupt**: Agent pauses before executing sensitive tool
2. **State Persistence**: Must use `checkpointer` and `thread_id`
3. **Resume**: Use `Command(resume={...})` to continue
4. **Decisions**: `[{"type": "approve"}]` or `[{"type": "reject"}]`

---

## Lab 1.1: Create Agent with HumanInTheLoopMiddleware

In [ ]:
# Create checkpointer for state persistence
checkpointer = InMemorySaver()

# Create agent with HITL middleware
hr_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=[
        get_employee_info,
        search_employees,
        update_salary,
        promote_employee,
        terminate_employee
    ],
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                # Safe operations - auto-approve
                "get_employee_info": False,
                "search_employees": False,
                
                # Sensitive operations - require approval
                "update_salary": True,
                "promote_employee": True,
                "terminate_employee": True,
            }
        )
    ],
    checkpointer=checkpointer,  # Required for HITL!
    prompt="""You are an HR assistant that helps with employee operations.
    
    Some operations require manager approval before execution.
    Be helpful and professional."""
)

print("✅ HR Agent with Human-in-the-Loop created!")
print("\n⚙️  Configuration:")
print("   Auto-approve: get_employee_info, search_employees")
print("   Require approval: update_salary, promote_employee, terminate_employee")

## Lab 1.2: Test Safe Operation (Auto-Approved)

In [ ]:
print("=" * 70)
print("TEST 1: Safe Operation (No Approval Needed)")
print("=" * 70)

config = {"configurable": {"thread_id": "test_safe_1"}}

result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "Tell me about employee 101"}]},
    config=config
)

print("\n🤖 Agent Response:")
print(result['messages'][-1].content)

print("\n✅ Operation completed immediately (no approval required)")

## Lab 1.3: Test Sensitive Operation (Requires Approval)

In [ ]:
print("=" * 70)
print("TEST 2: Sensitive Operation (Approval Required)")
print("=" * 70)

config = {"configurable": {"thread_id": "test_approval_1"}}

# Step 1: Initial request (will pause)
print("\nStep 1: User requests salary update...")
result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "Update salary for employee 101 to 150000"}]},
    config=config
)

print("\n⏸️  Agent Status: PAUSED (waiting for approval)")
print(f"\nAgent says: {result['messages'][-1].content}")

# Check if there's a pending approval
# In real implementation, you'd check agent state here

print("\n" + "="*70)
print("Step 2: Manager approves the operation...")
print("="*70)

# Step 2: Resume with approval
result = hr_agent.invoke(
    Command(resume={"decisions": [{"type": "approve"}]}),
    config=config  # Same thread_id!
)

print("\n✅ Agent Status: RESUMED")
print(f"\n🤖 Agent Response: {result['messages'][-1].content}")

print("\n✅ Operation completed after approval!")

## Lab 1.4: Test Rejection

In [ ]:
print("=" * 70)
print("TEST 3: Rejection Scenario")
print("=" * 70)

config = {"configurable": {"thread_id": "test_rejection_1"}}

# Step 1: Request termination (will pause)
print("\nStep 1: User requests employee termination...")
result = hr_agent.invoke(
    {"messages": [{"role": "user", "content": "Terminate employee 102"}]},
    config=config
)

print("\n⏸️  Agent Status: PAUSED")
print(f"\nAgent says: {result['messages'][-1].content}")

print("\n" + "="*70)
print("Step 2: Manager REJECTS the operation...")
print("="*70)

# Step 2: Resume with rejection
result = hr_agent.invoke(
    Command(resume={"decisions": [{"type": "reject"}]}),
    config=config
)

print("\n❌ Operation REJECTED")
print(f"\n🤖 Agent Response: {result['messages'][-1].content}")

print("\n✅ Agent handled rejection gracefully!")

---
# Part 2: Building Gradio UI for Approvals

Now let's create an interactive UI where managers can approve/reject operations in real-time!

---

## Lab 2.1: Gradio App - Simple Version

In [ ]:
# Global state for managing pending approvals
PENDING_APPROVALS = {}
THREAD_COUNTER = 0

def process_hr_request(user_request, thread_id=None):
    """Process HR request and check if approval needed."""
    global THREAD_COUNTER, PENDING_APPROVALS
    
    # Generate thread ID if not provided
    if not thread_id:
        THREAD_COUNTER += 1
        thread_id = f"thread_{THREAD_COUNTER}"
    
    config = {"configurable": {"thread_id": thread_id}}
    
    try:
        # Invoke agent
        result = hr_agent.invoke(
            {"messages": [{"role": "user", "content": user_request}]},
            config=config
        )
        
        response = result['messages'][-1].content
        
        # Check if agent is waiting for approval
        # (In real implementation, check state for interrupts)
        if "approval" in response.lower() or "requires" in response.lower():
            # Store as pending
            PENDING_APPROVALS[thread_id] = {
                "request": user_request,
                "response": response,
                "timestamp": datetime.now().isoformat()
            }
            return f"⏸️  **APPROVAL REQUIRED**\n\n{response}\n\n**Thread ID:** `{thread_id}`", thread_id
        else:
            return f"✅ **COMPLETED**\n\n{response}", thread_id
            
    except Exception as e:
        return f"❌ Error: {str(e)}", thread_id

def approve_operation(thread_id):
    """Approve pending operation."""
    if not thread_id:
        return "❌ No thread ID provided"
    
    config = {"configurable": {"thread_id": thread_id}}
    
    try:
        # Resume with approval
        result = hr_agent.invoke(
            Command(resume={"decisions": [{"type": "approve"}]}),
            config=config
        )
        
        response = result['messages'][-1].content
        
        # Remove from pending
        if thread_id in PENDING_APPROVALS:
            del PENDING_APPROVALS[thread_id]
        
        return f"✅ **APPROVED & EXECUTED**\n\n{response}"
        
    except Exception as e:
        return f"❌ Error during approval: {str(e)}"

def reject_operation(thread_id):
    """Reject pending operation."""
    if not thread_id:
        return "❌ No thread ID provided"
    
    config = {"configurable": {"thread_id": thread_id}}
    
    try:
        # Resume with rejection
        result = hr_agent.invoke(
            Command(resume={"decisions": [{"type": "reject"}]}),
            config=config
        )
        
        response = result['messages'][-1].content
        
        # Remove from pending
        if thread_id in PENDING_APPROVALS:
            del PENDING_APPROVALS[thread_id]
        
        return f"❌ **REJECTED**\n\n{response}"
        
    except Exception as e:
        return f"❌ Error during rejection: {str(e)}"

def get_pending_approvals():
    """Get list of pending approvals."""
    if not PENDING_APPROVALS:
        return "No pending approvals"
    
    output = "**Pending Approvals:**\n\n"
    for thread_id, info in PENDING_APPROVALS.items():
        output += f"🔸 **Thread:** `{thread_id}`\n"
        output += f"   Request: {info['request']}\n"
        output += f"   Time: {info['timestamp']}\n\n"
    
    return output

print("✅ Helper functions created!")

## Lab 2.2: Launch Gradio App

In [ ]:
# Create Gradio interface
with gr.Blocks(title="HR Human-in-the-Loop System", theme=gr.themes.Soft()) as demo:
    gr.Markdown("""
    # 👤 HR Human-in-the-Loop Approval System
    
    This system requires manager approval for sensitive HR operations:
    - 💰 Salary updates
    - 🎯 Promotions
    - ⛔ Terminations
    """)
    
    with gr.Tab("🔷 Make Request"):
        gr.Markdown("### Submit HR Operation Request")
        
        with gr.Row():
            with gr.Column():
                request_input = gr.Textbox(
                    label="HR Request",
                    placeholder="e.g., Update salary for employee 101 to 150000",
                    lines=3
                )
                submit_btn = gr.Button("Submit Request", variant="primary")
            
            with gr.Column():
                request_output = gr.Textbox(label="Response", lines=8)
                thread_id_output = gr.Textbox(label="Thread ID (save this!)", interactive=False)
        
        gr.Markdown("""
        **Example requests:**
        - `Tell me about employee 101` (safe, auto-approved)
        - `Update salary for employee 102 to 200000` (requires approval)
        - `Promote employee 103 to Chief HR Officer` (requires approval)
        - `Terminate employee 101` (requires approval)
        """)
    
    with gr.Tab("✅ Approve/Reject"):
        gr.Markdown("### Manager Approval Panel")
        
        with gr.Row():
            with gr.Column():
                pending_display = gr.Textbox(
                    label="Pending Approvals",
                    lines=6,
                    interactive=False
                )
                refresh_btn = gr.Button("🔄 Refresh Pending List")
            
            with gr.Column():
                thread_id_input = gr.Textbox(
                    label="Thread ID to Approve/Reject",
                    placeholder="Paste thread ID here"
                )
                
                with gr.Row():
                    approve_btn = gr.Button("✅ Approve", variant="primary")
                    reject_btn = gr.Button("❌ Reject", variant="stop")
                
                approval_output = gr.Textbox(label="Result", lines=6)
    
    with gr.Tab("📊 History"):
        gr.Markdown("### Approval History")
        history_output = gr.JSON(label="Approval History", value=APPROVAL_HISTORY)
        refresh_history_btn = gr.Button("🔄 Refresh History")
    
    # Event handlers
    submit_btn.click(
        fn=process_hr_request,
        inputs=[request_input],
        outputs=[request_output, thread_id_output]
    )
    
    approve_btn.click(
        fn=approve_operation,
        inputs=[thread_id_input],
        outputs=[approval_output]
    )
    
    reject_btn.click(
        fn=reject_operation,
        inputs=[thread_id_input],
        outputs=[approval_output]
    )
    
    refresh_btn.click(
        fn=get_pending_approvals,
        outputs=[pending_display]
    )
    
    refresh_history_btn.click(
        fn=lambda: APPROVAL_HISTORY,
        outputs=[history_output]
    )

# Launch the app
print("\n🚀 Launching Gradio app...\n")
demo.launch(share=True, debug=True)

---
# Part 3: Testing the Complete Flow

## How to Use the Gradio App:

### Step 1: Make a Request
1. Go to **"🔷 Make Request"** tab
2. Enter: `Update salary for employee 101 to 150000`
3. Click **"Submit Request"**
4. **Save the Thread ID** that appears!

### Step 2: Approve/Reject
1. Go to **"✅ Approve/Reject"** tab
2. Click **"🔄 Refresh Pending List"** to see pending requests
3. Paste the **Thread ID** from Step 1
4. Click **"✅ Approve"** or **"❌ Reject"**

### Step 3: View History
1. Go to **"📊 History"** tab
2. Click **"🔄 Refresh History"**
3. See all approved operations!

---

## Lab 3.1: Programmatic Testing (Without UI)

In [ ]:
print("=" * 70)
print("COMPLETE WORKFLOW TEST")
print("=" * 70)

# Scenario: Promote an employee
print("\n📝 Scenario: Promote employee 101 to Lead Developer")
print("="*70)

response, thread_id = process_hr_request("Promote employee 101 to Lead Developer")
print(f"\nAgent Response:\n{response}")
print(f"\nThread ID: {thread_id}")

print("\n" + "="*70)
print("👤 Manager reviews and approves...")
print("="*70)

approval_result = approve_operation(thread_id)
print(f"\n{approval_result}")

print("\n" + "="*70)
print("📊 Approval History")
print("="*70)
print(json.dumps(APPROVAL_HISTORY, indent=2))

print("\n✅ Complete workflow tested successfully!")

---
# Advanced: Enhanced Gradio App with Rich Features

Let's create a more sophisticated version with:
- Real-time status updates
- Detailed approval context
- Approval notes/comments
- Multi-approver support

---

## Lab 4: Enhanced Gradio App

In [ ]:
# Enhanced approval system
class ApprovalSystem:
    def __init__(self):
        self.pending = {}
        self.history = []
        self.counter = 0
    
    def create_request(self, user_request, agent_response, thread_id):
        """Create new approval request."""
        self.pending[thread_id] = {
            "id": thread_id,
            "request": user_request,
            "response": agent_response,
            "created_at": datetime.now().isoformat(),
            "status": "pending"
        }
    
    def approve(self, thread_id, approver_name, notes=""):
        """Approve request."""
        if thread_id in self.pending:
            request = self.pending[thread_id]
            request["status"] = "approved"
            request["approver"] = approver_name
            request["notes"] = notes
            request["decided_at"] = datetime.now().isoformat()
            
            self.history.append(request)
            del self.pending[thread_id]
            return True
        return False
    
    def reject(self, thread_id, approver_name, notes=""):
        """Reject request."""
        if thread_id in self.pending:
            request = self.pending[thread_id]
            request["status"] = "rejected"
            request["approver"] = approver_name
            request["notes"] = notes
            request["decided_at"] = datetime.now().isoformat()
            
            self.history.append(request)
            del self.pending[thread_id]
            return True
        return False
    
    def get_pending_list(self):
        """Get formatted list of pending requests."""
        if not self.pending:
            return "✅ No pending approvals"
        
        output = "## 📋 Pending Approvals\n\n"
        for thread_id, req in self.pending.items():
            output += f"### 🔸 Request ID: `{thread_id}`\n"
            output += f"**Request:** {req['request']}\n"
            output += f"**Created:** {req['created_at']}\n"
            output += f"**Status:** ⏸️ Waiting for approval\n\n"
            output += "---\n\n"
        return output
    
    def get_history(self):
        """Get approval history."""
        return self.history

# Create global approval system
approval_system = ApprovalSystem()

print("✅ Enhanced approval system created!")

In [ ]:
# Enhanced helper functions
def enhanced_process_request(user_request):
    """Process request with enhanced tracking."""
    approval_system.counter += 1
    thread_id = f"req_{approval_system.counter}"
    
    config = {"configurable": {"thread_id": thread_id}}
    
    try:
        result = hr_agent.invoke(
            {"messages": [{"role": "user", "content": user_request}]},
            config=config
        )
        
        response = result['messages'][-1].content
        
        # Check if approval needed
        if "approval" in response.lower() or "requires" in response.lower():
            approval_system.create_request(user_request, response, thread_id)
            return f"⏸️  **APPROVAL REQUIRED**\n\n{response}\n\n---\n**Thread ID:** `{thread_id}`\n\nPlease go to the Approve/Reject tab to process this request.", thread_id
        else:
            return f"✅ **COMPLETED IMMEDIATELY**\n\n{response}", thread_id
            
    except Exception as e:
        return f"❌ **ERROR:** {str(e)}", ""

def enhanced_approve(thread_id, approver_name, notes):
    """Enhanced approval with notes."""
    if not thread_id:
        return "❌ Please enter a Thread ID"
    
    if not approver_name:
        return "❌ Please enter your name"
    
    config = {"configurable": {"thread_id": thread_id}}
    
    try:
        # Resume with approval
        result = hr_agent.invoke(
            Command(resume={"decisions": [{"type": "approve"}]}),
            config=config
        )
        
        # Record approval
        approval_system.approve(thread_id, approver_name, notes)
        
        response = result['messages'][-1].content
        return f"✅ **APPROVED BY:** {approver_name}\n\n**Notes:** {notes if notes else 'None'}\n\n---\n\n{response}"
        
    except Exception as e:
        return f"❌ **ERROR:** {str(e)}"

def enhanced_reject(thread_id, approver_name, notes):
    """Enhanced rejection with notes."""
    if not thread_id:
        return "❌ Please enter a Thread ID"
    
    if not approver_name:
        return "❌ Please enter your name"
    
    if not notes:
        return "❌ Please provide a reason for rejection"
    
    config = {"configurable": {"thread_id": thread_id}}
    
    try:
        # Resume with rejection
        result = hr_agent.invoke(
            Command(resume={"decisions": [{"type": "reject"}]}),
            config=config
        )
        
        # Record rejection
        approval_system.reject(thread_id, approver_name, notes)
        
        response = result['messages'][-1].content
        return f"❌ **REJECTED BY:** {approver_name}\n\n**Reason:** {notes}\n\n---\n\n{response}"
        
    except Exception as e:
        return f"❌ **ERROR:** {str(e)}"

print("✅ Enhanced helper functions ready!")

In [ ]:
# Launch enhanced Gradio app
with gr.Blocks(title="HR Approval System - Enhanced", theme=gr.themes.Soft()) as enhanced_demo:
    gr.Markdown("""
    # 👥 HR Human-in-the-Loop Approval System (Enhanced)
    
    Professional approval workflow for sensitive HR operations with full audit trail.
    """)
    
    with gr.Tab("📝 Submit Request"):
        gr.Markdown("### Employee/HR Request Submission")
        
        with gr.Row():
            with gr.Column(scale=1):
                req_input = gr.Textbox(
                    label="HR Operation Request",
                    placeholder="Describe the HR operation...",
                    lines=4
                )
                submit_enhanced_btn = gr.Button("📤 Submit Request", variant="primary", size="lg")
                
                gr.Markdown("""
                **Quick Examples:**
                ```
                Update salary for employee 101 to 160000
                Promote employee 102 to Senior Manager
                Terminate employee 103
                ```
                """)
            
            with gr.Column(scale=1):
                req_output = gr.Markdown(label="Response")
                thread_id_display = gr.Textbox(label="🔑 Thread ID (Copy This!)", interactive=False)
    
    with gr.Tab("✅ Manager Approval"):
        gr.Markdown("### Approval Dashboard")
        
        with gr.Row():
            with gr.Column():
                pending_list = gr.Markdown(value="Click refresh to load pending requests")
                refresh_pending_btn = gr.Button("🔄 Refresh Pending List", size="sm")
            
            with gr.Column():
                gr.Markdown("### Decision Panel")
                
                thread_input = gr.Textbox(label="Thread ID", placeholder="Paste thread ID")
                approver_input = gr.Textbox(label="Your Name (Approver)", placeholder="e.g., John Doe")
                notes_input = gr.Textbox(
                    label="Notes/Comments",
                    placeholder="Optional: Add approval notes or reason for rejection",
                    lines=3
                )
                
                with gr.Row():
                    approve_enhanced_btn = gr.Button("✅ Approve", variant="primary")
                    reject_enhanced_btn = gr.Button("❌ Reject", variant="stop")
                
                decision_output = gr.Markdown()
    
    with gr.Tab("📊 Audit Trail"):
        gr.Markdown("### Complete Approval History")
        history_display = gr.JSON(label="Approval History")
        refresh_hist_btn = gr.Button("🔄 Refresh History")
    
    # Wire up events
    submit_enhanced_btn.click(
        fn=enhanced_process_request,
        inputs=[req_input],
        outputs=[req_output, thread_id_display]
    )
    
    refresh_pending_btn.click(
        fn=lambda: approval_system.get_pending_list(),
        outputs=[pending_list]
    )
    
    approve_enhanced_btn.click(
        fn=enhanced_approve,
        inputs=[thread_input, approver_input, notes_input],
        outputs=[decision_output]
    )
    
    reject_enhanced_btn.click(
        fn=enhanced_reject,
        inputs=[thread_input, approver_input, notes_input],
        outputs=[decision_output]
    )
    
    refresh_hist_btn.click(
        fn=lambda: approval_system.get_history(),
        outputs=[history_display]
    )

print("\n🚀 Launching ENHANCED Gradio app...\n")
enhanced_demo.launch(share=True, debug=True)

---
# Summary

## Key Learnings

### HumanInTheLoopMiddleware Configuration
```python
HumanInTheLoopMiddleware(
    interrupt_on={
        "safe_tool": False,      # Auto-approve
        "sensitive_tool": True,  # Require approval
    }
)
```

### Critical Requirements
- ✅ Must use `checkpointer` for state persistence
- ✅ Must provide `thread_id` in config
- ✅ Use `Command(resume={...})` to resume
- ✅ Same `thread_id` to resume paused conversation

### Decision Types
- `{"type": "approve"}` - Execute the operation
- `{"type": "reject"}` - Cancel the operation

### Gradio Integration Benefits
- 🎨 Visual approval workflow
- 👥 Multi-user support
- 📊 Real-time status tracking
- 📝 Approval notes and audit trail
- 🔄 Easy resumption of paused operations

## Production Checklist

- [ ] Identify tools requiring approval
- [ ] Configure `interrupt_on` dictionary
- [ ] Set up persistent checkpointer (SQLite, Postgres)
- [ ] Build approval UI (Gradio, Web app, Slack bot)
- [ ] Implement notification system
- [ ] Add audit logging
- [ ] Set approval timeouts
- [ ] Multi-level approval chains
- [ ] Role-based access control

---

**Congratulations!** 🎉 You now have a complete Human-in-the-Loop system with interactive Gradio UI!